# Telegram-бот

## Создание бота

Сначала создадим наш бот. Для этого найдем в Telegram бот для создания ботов: [@BotFather](https://t.me/BotFather). По кнопке Старт или команде `/start` запускаем и проходим шаги, которые требует от нас бот: придумываем название бота, затем имя пользователя. После успешного создания мы получаем длинное сообщение, начинающееся со слов "Done! Congratulations on your new bot." Во втором абзаце этого сообщения содержится токен. Он нам понадобится, чтобы привязать нашего бота к нашему коду.

## Установка библиотек

Теперь нам понадобится библиотека, которая обеспечивает возможность работы с Telegram через Python. Такие библиотеки называются API (Application Programming Interface). Мы будем пользоваться библиотекой `pyTelegramBotAPI`. Она не предустановлена, поэтому надо будет ее установить. Запускаем терминал (Mac), консоль (Linux) или командную строку (Windows) и пишем команду для установки:

```
pip install pyTelegramBotAPI
```

Если не сработало, пробуем по-другому:

```
pip3 install pyTelegramBotAPI
```

Если компьютер считает, что ни pip, ни pip3 у вас не установлены, нужно сначала установить pip, а потом уже библиотеку:

```
python -m pip install
pip install pyTelegramBotAPI
```

или

```
python3 -m pip install
pip3 install pyTelegramBotAPI
```

Варианты pip/pip3 и python/python3 зависят просто от того, какая версия Python у вас установлена.

## Обработка команд

После того, как установили библиотеку, можем ею пользоваться. Для этого в начале нашего кода импортируем библиотеку: `import telebot`. Затем нам нужно привязать приложение к боту, которого мы создали. Копируем из BotFather токен, который получили при создании бота и в этом коде вставляем его вместо слова TOKEN: `bot = telebot.TeleBot('TOKEN')`. В этой строчке важно капитализация (где заглавные буквы, а где - строчные) и заключение токена в кавычки.

Перепишите себе код ниже, не забудьте вставить свой токен. Прочитайте комментарии, постарайтесь понять, что происходит в каждой строчке. Запустите получившееся приложение, протестируйте его работу (в Telegram отправьте боту команду `/start`).

```python
import telebot # Импортируем библиотеку

bot = telebot.TeleBot('TOKEN') # Привязываем приложение к боту

@bot.message_handler(commands=['start']) # Создаем слушатель сообщений для команды /start
def start_command(message): # Функция, которая будет запущена при получении команды. 
                            # Функция имеет параметр message, в нее будет подставлено сообщение от пользователя
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')# Метод библиотеки, отправляющий 
                                                                      # сообщение. Первый аргумент 
                                                                      # определяет, в какой чат отправлять,
                                                                      # второй содержит текст сообщения.

bot.polling(none_stop=True) # Эта строка нужна для того, чтобы приложение не завершало свою работу
```

У нас получился бот, который умеет получать команду `/start` и отвечать на нее приветственным сообщением.

Теперь добавим простую функцию, которая работает с сообщением от пользователя: получив сообщение, она выведет в консоль IDLE текст этого сообщения. Ниже - полный код приложения, которое должно получиться. Код после любого изменения запускаем и проверяем корректность работы. 

```python
import telebot

bot = telebot.TeleBot('TOKEN')

@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')

@bot.message_handler(func=lambda m: True) # Новый слушатель будет ловить не команды, а сообщения
def handle_message(message):
    print(message.text) # Берем полученное сообщение, находим у него поле text и печатаем содержимое

bot.polling(none_stop=True)

```

### Задание

В коде бота, который печатает в консоли текст полученного сообщения, удалите `.text`, чтобы функция, обрабатывающая сообщение, выполняла `print(message)`. Запустите, отправьте боту сообщение, посмотрите, что будет выведено в консоль. Что это за тип данных? Какие данные мы получаем с каждым сообщением от пользователя? Допишите код так, чтобы при получении сообщения от пользователя бот выводил в консоль имя пользователя (username). Подсказка: структура полученных данных не однородна, содержит вложенность.

## Кнопки

Теперь добавим возможность общаться с ботом при помощи нажатия кнопок.

```python
import telebot

bot = telebot.TeleBot('TOKEN')

@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')

@bot.message_handler(func=lambda m: True) # Клавиатуру с кнопками будем отправлять в ответ 
                                          # на сообщение пользователя
def handle_message(message):
    markup = telebot.types.ReplyKeyboardMarkup() # Сначала создадим клавиатуру, 
                                                 # в которой будут содержаться наши кнопки.
    item_1 = telebot.types.InlineKeyboardButton(text='Кнопка 1') # Создадим первую кнопку
    item_2 = telebot.types.InlineKeyboardButton(text='Кнопка 2') # Создадим вторую кнопку
    markup.add(item_1, item_2) # Добавим обе созданные кнопки в созданную клавиатуру
    bot.send_message(message.chat.id, 'Выбери кнопку', reply_markup=markup) # Отправим пользователю сообщение 
                                                                            # с возможностью ответить на него 
                                                                            # при помощи кнопок.

bot.polling(none_stop=True)

```

Для того, чтобы кнопки занимали ровно столько места, сколько им нужно, можем при создании клавиатуры добавить соответствующий аргумент:

```python
markup = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True)
```

Есть другой вид клавиатуры. Делаем все то же самое, только вместо `ReplyKeyboardMarkup` пишем `InlineKeyboardMarkup`. Тогда кнопки будут не под строкой ввода сообщения, а под сообщением от бота. С этим видом кнопок аргумент `resize_keyboard=True` не работает (он и не нужен), скобки при создании клавиатуры нужно оставлять пустыми. Для этого вида клавиатуры обязателен аргумент `callback_data`, который определяет действие, выполняемое по нажатию кнопки.

```python
item = telebot.types.InlineKeyboardButton(text='Кнопка 1',
                                              callback_data='button_1')
```

Запустите получившийся бот и посмотрите, как изменились кнопки.

Все хорошо, только эти кнопки не работают. На самом деле не так. Они работают, просто мы не прописали в приложении действия, которые должны выполняться по нажатию на эти кнопки. Для этого введем новый слушатель, только в этот раз он будет ждать не сообщение, а конкретную команду с кнопки.

```python
import telebot

bot = telebot.TeleBot('TOKEN')

@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')

@bot.message_handler(func=lambda m: True)
def handle_message(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Кнопка 1', callback_data='button_1')
    item_2 = telebot.types.InlineKeyboardButton(text='Кнопка 2', callback_data='button_2')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Выбери кнопку', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'button_1') # Слушатель команды - колбэка
def button_1(call): # call содержит все данные, которые мы получим от кнопки
    print(call) # Напечатаем эти данные и посмотрим, что в них содержится
    bot.send_message(call.message.chat.id, 'Вы нажали кнопку 1') # И отправим пользователю сообщение

bot.polling(none_stop=True)

```

Код ниже предлагает выбрать тему для получения ссылки на конспект этой темы.

```python
import telebot

bot = telebot.TeleBot(TOKEN)

link_1 = {'name': 'Ввод-вывод, математические операции', 'link': 'https://github.com/naleont/8-III/blob/main/01_input-output_math-operations.ipynb'}
link_2 = {'name': 'Строки', 'link': 'https://github.com/naleont/8-III/blob/main/02_strings.ipynb'}
link_3 = {'name': 'Условный оператор', 'link': 'https://github.com/naleont/8-III/blob/main/03_conditions.ipynb'}


@bot.message_handler(commands=['start'])
def start_command(message):
    markup = telebot.types.InlineKeyboardMarkup(row_width=1) # Добавлено максимальное число кнопок в ряду
    item_1 = telebot.types.InlineKeyboardButton(text='Ввод-вывод, математические операции', callback_data='theory_1')
    item_2 = telebot.types.InlineKeyboardButton(text='Строки', callback_data='theory_2')
    item_3 = telebot.types.InlineKeyboardButton(text='Условный оператор', callback_data='theory_3')
    markup.add(item_1, item_2, item_3)
    bot.send_message(message.chat.id, 'Выбери тему', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'theory_1')
def theory_1(call):
    name = link_1['name']
    link = link_1['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

@bot.callback_query_handler(func=lambda call: call.data == 'theory_2')
def theory_2(call):
    name = link_2['name']
    link = link_2['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

@bot.callback_query_handler(func=lambda call: call.data == 'theory_3')
def theory_3(call):
    name = link_3['name']
    link = link_3['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

bot.polling(none_stop=True)
```



А это - тот же бот, написанный более эффективным кодом:

```python
import telebot

bot = telebot.TeleBot(TOKEN)

# Объединили данные по всей теории в список вместо использования отдельной переменной для каждой темы
links = [{'id': '1', 'name': 'Ввод-вывод, математические операции', 'link': 'https://github.com/naleont/8-III/blob/main/01_input-output_math-operations.ipynb'},
{'id': '2', 'name': 'Строки', 'link': 'https://github.com/naleont/8-III/blob/main/02_strings.ipynb'},
{'id': '3', 'name': 'Условный оператор', 'link': 'https://github.com/naleont/8-III/blob/main/03_conditions.ipynb'},
{'id': '4', 'name': 'Больше об условиях и булевых данных', 'link': 'https://github.com/naleont/8-III/blob/main/04_more-conditions_boolean-type.ipynb'},
{'id': '5', 'name': 'Списки, цикл while', 'link': 'https://github.com/naleont/8-III/blob/main/05_lists_while-loop.ipynb'},
{'id': '6', 'name': 'Цикл for, словари', 'link': 'https://github.com/naleont/8-III/blob/main/06_for-loop_dictionaries.ipynb'},
{'id': '7', 'name': 'Функции, файлы', 'link': 'https://github.com/naleont/8-III/blob/main/07_files_functions.ipynb'}]


@bot.message_handler(commands=['start'])
def start_command(message):
    markup = telebot.types.InlineKeyboardMarkup(row_width=1)
    # Для получения данных по всем темам проходимся циклом по списку
    for link in links:
        # ID темы добавляем к имени колбэка
        item = telebot.types.InlineKeyboardButton(text=link['name'], callback_data='theory_' + link['id'])
        markup.add(item)
    bot.send_message(message.chat.id, 'Выбери тему', reply_markup=markup)

# В слушателе ловим только часть колбэка, одинаковую для всех тем
@bot.callback_query_handler(func=lambda call: call.data.split('_')[0] == 'theory')
def button_1(call):
    # Конкретную тему определяем по второй части колбэка
    link_id = call.data.split('_')[1]
    # Фильтруем список тем по ID нужной темы
    link_data = list(filter(lambda x: x['id'] == link_id, links))[0]
    name = link_data['name']
    link = link_data['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

bot.polling(none_stop=True)
```

Еще полезные штуки:

```python
import telebot

bot = telebot.TeleBot(TOKEN)


def birthdate_reply(message):
    birthdate = message.text
    print(birthdate)


@bot.message_handler(commands=['start'])
def start_command(message):
    # ForceReply сделает, чтобы следующее сообщение от пользователя использовало цитирование сообщения от бота
    # input_field_placeholder добавит пример ввода данных
    markup = telebot.types.ForceReply(input_field_placeholder='01.01.1990', selective=False)
    msg = bot.send_message(message.chat.id, 'Введите вашу дату рождения', reply_markup=markup)
    # register_next_step_handler определит, какая функция должна быть запущена при получении следующего сообщения пользоватля
    bot.register_next_step_handler(msg, birthdate_reply)

bot.polling(none_stop=True)
```

А это бот с опросом. [Здесь](https://github.com/naleont/8-III/blob/main/09_tg-bots.pdf) - блок-схема с логикой работы. Сначала изучите ее, а потом - код ниже.

```python

import telebot

bot = telebot.TeleBot(TOKEN)

data = {}

def school_student(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Да', callback_data='school_student-yes')
    item_2 = telebot.types.InlineKeyboardButton(text='Нет', callback_data='school_student-no')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Вы учитесь в школе?', reply_markup=markup)


def school_class(message):
    markup = telebot.types.ForceReply(input_field_placeholder='10', selective=False)
    bot.register_next_step_handler(message, school_class_handler)
    bot.send_message(message.chat.id, 'В каком вы классе?', reply_markup=markup)


def school_class_handler(message):
    data['school_class'] = message.text
    good_sleep(message)


def good_sleep(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Да', callback_data='good_sleep-yes')
    item_2 = telebot.types.InlineKeyboardButton(text='Нет', callback_data='good_sleep-no')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Вы сегодня выспались?', reply_markup=markup)


@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id,
    'Спасибо, что согласились принять участие в опросе! Он займет всего пару минут. Все данные хранятся анонимно.')
    school_student(message)


@bot.callback_query_handler(func=lambda call: call.data.split('-')[0] == 'school_student')
def school_student_handler(call):
    reply = call.data.split('-')[1]
    data['school_student'] = reply
    if reply == 'yes':
        school_class(call.message)
    else:
        data['school_class'] = ''
        good_sleep(call.message)


@bot.callback_query_handler(func=lambda call: call.data.split('-')[0] == 'good_sleep')
def good_sleep_handler(call):
    reply = call.data.split('-')[1]
    data['good_sleep'] = reply
    if reply == 'yes':
        bot.send_message(call.message.chat.id, 'Здорово!')
    else:
        bot.send_message(call.message.chat.id, 'Сон - это важно! Не забудьте выспаться сегодня!')
    bot.send_message(call.message.chat.id, 'Опрос завершен, спасибо за участие!')

bot.polling(none_stop=True)
```

Это далеко не все возможности библиотеки. Если хотите использовать другие возможности, изучите ее **[документацию](https://pytba.readthedocs.io/en/latest/types.html#telebot.types.InlineKeyboardMarkup.max_row_keys)**. 